In [10]:
from astropy.cosmology import FlatLambdaCDM
from time import clock
from os import path
import glob, os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.miles_util as lib
from astropy.table import Table, Column

In [2]:
def flux2L(flux,z):
    """Transfer flux to luminoity assuming a flat Universe"""
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    DL = cosmo.luminosity_distance(z).value*10**6*3.08*10**18 # unit cm
    L = flux*1.e-17*4.*np.pi*DL**2   #erg/s/A
    return L

def L_conti(wave,flux,z):
    return np.asarray([np.log10(5100*flux2L(flux[np.where( abs(wave - 5100.) < 5.,True,False)].mean(),z) )])

In [5]:
tie_balmer=False
limit_doublets=False
ppxf_dir = path.dirname(path.realpath(ppxf_package.__file__))

mass_frac_all = np.array([])
ml_all = np.array([])
L_all = np.array([])
zbin1_syst_vel = []

In [6]:
for i in zip(glob.glob('/Users/ANguyen\MyWorks\zbin1_specs_sv\spec-*.fits')):
    #print (np.shape(i))
    hdu = fits.open(i[0])
    t = hdu[1].data
    wv = 10**t['loglam']
    redshift = float(hdu[2].data['z'])
    #print ('redshift:',np.round(redshift,4))
    
    wave1 = 10**t['loglam']/(1.+ redshift)
    flux1 = t['flux']
    err1 = 1./np.sqrt(t['ivar'])
    z = 0.
    mask = np.where( (wave1 > 3541.) & (wave1 < 7409.) & (err1 <100.), True, False)

    flux = flux1[mask]
    galaxy = flux/np.median(flux)   # Normalize spectrum to avoid numerical issues
    factor1=np.median(flux)
    #print 'flux normalized by:', np.median(flux)
    wave = wave1[mask]
    err = err1[mask]
    wave *= np.median(util.vac_to_air(wave)/wave)
    noise = np.full_like(galaxy, 0.01635)  # Assume constant noise per pixel here
    c = 299792.458  # speed of light in km/s
    velscale = c*np.log(wave[1]/wave[0])  # eq.(8) of Cappellari (2017)
    FWHM_gal = 2.76  # SDSS has an approximate instrumental resolution FWHM of 2.76A.
    pathname = ppxf_dir + '/miles_models/Mun1.30*.fits'
    miles = lib.miles(pathname, velscale, FWHM_gal)
    reg_dim = miles.templates.shape[1:]
    stars_templates = miles.templates.reshape(miles.templates.shape[0], -1)

    regul_err = 0.01  # Desired regularization error

    lam_range_gal = np.array([np.min(wave), np.max(wave)])/(1. + z)
    gas_templates, gas_names, line_wave = util.emission_lines(
        miles.log_lam_temp, lam_range_gal, FWHM_gal,
        tie_balmer=tie_balmer, limit_doublets=limit_doublets)
    templates = np.column_stack([stars_templates, gas_templates])

    c = 299792.458
    dv = c*(miles.log_lam_temp[0] - np.log(wave[0]))  # eq.(8) of Cappellari (2017)
    vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
    start = [vel, 180.]  # (km/s), starting guess for [V, sigma]

    n_temps = stars_templates.shape[1]
    n_forbidden = np.sum(["[" in a for a in gas_names])  # forbidden lines contain "[*]"
    n_balmer = len(gas_names) - n_forbidden
    component = [0]*n_temps + [1]*n_balmer + [2]*n_forbidden
    gas_component = np.array(component) > 0  # gas_component=True for gas templates
    moments = [4, 4, 4]

    start = [start, start, start]

    gas_reddening = 0 if tie_balmer else None
    t = clock()
    
    len_galaxy = len(galaxy)
    len_template = len(templates)
    difference = len_galaxy - len_template

    if len_galaxy > len_template:
        galaxy = galaxy[:len(galaxy)-difference]
        noise = noise[:len(noise)-difference]
        wave = noise[:len(wave)-difference]
        
    elif len_galaxy == len_template:
        galaxy = galaxy[:len(galaxy)-1]
        noise = noise[:len(noise)-1]
        wave = noise[:len(wave)-1]
    else:
        galaxy = galaxy
        noise = noise
        
    pp = ppxf(templates, galaxy, noise, velscale, start,
              plot=False, moments=moments, degree=-1, mdegree=10, vsyst=dv,
              lam=wave, clean=False, regul=1./regul_err, reg_dim=reg_dim,
              component=component, gas_component=gas_component,
              gas_names=gas_names, gas_reddening=gas_reddening)
    
    syst_vel = c*np.log(1 + redshift) + pp.sol[1][0]
    
    print ('syst_vel',syst_vel)
    
    zbin1_syst_vel.append(syst_vel)
    
    #print('Desired Delta Chi^2: %.4g' % np.sqrt(2*galaxy.size))
    #print('Current Delta Chi^2: %.4g' % ((pp.chi2 - 1)*galaxy.size))

      

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -66        92    -0.004     0.074
 comp.  1:       -83       175     0.037     0.109
 comp.  2:       -32       172    -0.055     0.084
chi2/DOF: 12.32; degree = -1; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 187; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  28 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta       1.68     0.055     -83   175
Comp:  1        Hgamma      2.757     0.055     -83   175
Comp:  1         Hbeta      6.529     0.054     -83   175
Comp:  1        Halpha      29.97     0.053     -83   175
Comp:  2     [OII]3726       2.76     0.081     -32   172
Comp:  2     [OII]3729      10.24     0.081     -32   172
Comp:  2     [SII]6716      5.742     0.052     -32   172
Comp:  2     [SII]6731      5.523     0.052     -32   1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -101       167     0.089     0.112
 comp.  1:      -185        87     0.050     0.100
 comp.  2:      -147        96     0.079     0.138
chi2/DOF: 13.4; degree = -1; mdegree = 10
method = capfit; Jac calls: 10; Func calls: 236; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  29 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      1.165     0.043    -185    87
Comp:  1        Hgamma      2.329     0.043    -185    87
Comp:  1         Hbeta      5.402     0.042    -185    87
Comp:  1        Halpha      20.58      0.04    -185    87
Comp:  2     [OII]3726      1.615     0.056    -147    96
C

...

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       124       111     0.076    -0.036
 comp.  1:       185        86    -0.001     0.006
 comp.  2:       194        82     0.028     0.055
chi2/DOF: 9.145; degree = -1; mdegree = 10
method = capfit; Jac calls: 12; Func calls: 279; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  55 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.2876     0.041     185    86
Comp:  1        Hgamma     0.3476     0.041     185    86
Comp:  1         Hbeta     0.6002      0.04     185    86
Comp:  1        Halpha      3.442     0.038     185    86
Comp:  2     [OII]3726     0.6919     0.049     194    82


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        32       150     0.111     0.007
 comp.  1:        76       165     0.029     0.058
 comp.  2:       100       163    -0.053     0.062
chi2/DOF: 5.811; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 163; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  40 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.2554     0.052      76   165
Comp:  1        Hgamma     0.4484     0.052      76   165
Comp:  1         Hbeta      1.477     0.051      76   165
Comp:  1        Halpha      5.736     0.051      76   165
Comp:  2     [OII]3726       3.79     0.076     100   163
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -100       156    -0.007     0.030
 comp.  1:      -152       126     0.148     0.018
 comp.  2:      -151       104     0.106     0.062
chi2/DOF: 3.037; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 209; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  47 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta   0.001722     0.046    -152   126
Comp:  1        Hgamma     0.1575     0.046    -152   126
Comp:  1         Hbeta     0.1886     0.045    -152   126
Comp:  1        Halpha      1.316     0.044    -152   126
Comp:  2     [OII]3726          0     0.055    -151   104
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -168       140     0.054    -0.093
 comp.  1:      -174       147    -0.021    -0.052
 comp.  2:      -186       160    -0.082     0.076
chi2/DOF: 225.9; degree = -1; mdegree = 10
method = capfit; Jac calls: 10; Func calls: 233; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  37 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      1.145     0.047    -174   147
Comp:  1        Hgamma      1.364     0.047    -174   147
Comp:  1         Hbeta      2.202     0.046    -174   147
Comp:  1        Halpha      19.67     0.045    -174   147
Comp:  2     [OII]3726          0     0.076    -186   160


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       142       164     0.059     0.068
 comp.  1:       234       209    -0.259     0.103
 comp.  2:       164        97    -0.027     0.064
chi2/DOF: 7.883; degree = -1; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 186; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  44 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.1904     0.058     234   209
Comp:  1        Hgamma     0.7057     0.058     234   209
Comp:  1         Hbeta      1.421     0.057     234   209
Comp:  1        Halpha      5.889     0.056     234   209
Comp:  2     [OII]3726      3.477     0.053     164    97
C

In [2]:
import numpy as np
np.__version__

'1.16.2'

In [8]:
zbin1_syst_vel_array = np.array(zbin1_syst_vel)

In [11]:
col0 = fits.Column(name = 'SYST_VEL', format = 'D', array = zbin1_syst_vel_array)

coldefs = fits.ColDefs([col0])
hdu = fits.BinTableHDU.from_columns(coldefs)

zbin1_sv = Table(hdu.data)

In [13]:
# (zbin1_sv).write('zbin1_sv.fits')